# Scraper


In [1]:
# Construct my start_urls for all the cases.
# As there are no "previous/next" link betweem cases,
# we cannot use a typical recursive algorithm.
# Thus, we use the fact that the case names are structured:
# year + number + year
# This is surely not the best way!

last_year_in_data = 2018 # to be updated

years = list(range(2015, last_year_in_data+1))
years = [str(y) for y in years] # convert to string to concatenate with case numbers

case_num = list(range(1, 60))
# convert to string to concatenate with years:
case_num = ['-' + str(c).zfill(3) + 'N' for c in case_num] # zfill adds 0 to the left

# concatenate all string and years together:
cases = []
for y in years:
    for c in case_num:
        str(cases.append(y + c))

# complete URLs:
cases = ['https://www.ekr.admin.ch/f524/' + case + '.html#' + case for case in cases]


### Issues to check:

* 2014-005N -> several cases in history: 2015-048N, (and thus appears here?)
    * https://www.ekr.admin.ch/f524/2015-043N.html#2016-020N
    * https://www.ekr.admin.ch/prestations/f524/2014-005N.html?p=1 il y a bcp de texte...
    

* 2017-029N is missing
    * because no history? 


* 2016-020N missing too

* have duplicates: 2015-043N, 2015-047N, 2017-010N (4), 2018-009N, 2018-015N (3), 

* the accents are not read well by Excel. Excel issue?

In [2]:
# Importing in each cell because need to restart kernel
import scrapy # version 2.2.0
import time
from scrapy.crawler import CrawlerProcess

t0 = time.time()
print("Running...")

class CFRSpider(scrapy.Spider):
    name = "CFR" # Naming the spider if you  running more than one spider of this class simultaneously.

    # URL(s) to start with.
#    start_urls = [
#        'https://www.ekr.admin.ch/f524/2017-029N.html#2017-029N',
#        'https://www.ekr.admin.ch/f524/2018-036N.html#2018-036N',
#    ]
    start_urls = cases

    # What to do with the URL. 
    def parse(self, response):
        
        # Yield a dictionary with the values we want.
        yield {
            
            'case': response.xpath('body/div/div[2]/div/div/div/div/div/p/text()').extract_first(),
            'name': response.xpath('body/div/div[2]/div/div/div/div/div/h2/text()').extract_first(),
            'location': response.xpath('body/div/div[2]/div/div/div/div/div/p[2]/text()').extract_first(),
            
            # procedure history | Historique de la procédure | Verfahrensgeschichte | Cronistoria della procedura
            'year': response.xpath('body/div/div[2]/div/div/div/div/div/table[1]/tr/td/text()').extract()[0],
            'link': response.xpath('body/div/div[2]/div/div/div/div/div/table[1]/tr/td[2]/span/a/@href').extract_first(),
            'history': response.xpath('string(body/div/div[2]/div/div/div/div/div/table[1]/tr/td[3])').extract_first(),
            
            # keywords | Mots-clés | Stichwörter | Parole chiave
            'authors': response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[1]/td[2])').extract_first(),
            'victims': response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[2]/td[2])').extract_first(),
            'means': response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[3]/td[2])').extract_first(),
            'social_env': response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[4]/td[2])').extract_first(),
            'ideology': response.xpath('string(body/div/div[2]/div/div/div/div/div/table[3]/tr[5]/td[2])').extract_first(),
            
            # not sure yet what is the best for the text...
            # there is no new div, so it is not possible to distinguish it from the table above in the hierarchy
            # usually there are 2 elements: "Synthèse" and "Dècision", but sometimes more.
            # we will try to get them separately, both title h3 and text p,
            # with the idea to do some Pandas magic after
            # we think that there are no more than 5 titles, let's start with 6 to be sure
            'title_1': response.xpath('body/div/div[2]/div/div/div/div/div/h3[1]/text()').extract_first(),
            'title_2': response.xpath('body/div/div[2]/div/div/div/div/div/h3[2]/text()').extract_first(),
            'title_3': response.xpath('body/div/div[2]/div/div/div/div/div/h3[3]/text()').extract_first(),
            'title_4': response.xpath('body/div/div[2]/div/div/div/div/div/h3[4]/text()').extract_first(),
            'title_5': response.xpath('body/div/div[2]/div/div/div/div/div/h3[5]/text()').extract_first(),
            'title_6': response.xpath('body/div/div[2]/div/div/div/div/div/h3[6]/text()').extract_first(),
            # start at p[3], because the 2 first p are case and location
            'text_1': response.xpath('string(body/div/div[2]/div/div/div/div/div/p[3])').extract_first(),
            'text_2': response.xpath('string(body/div/div[2]/div/div/div/div/div/p[4])').extract_first(),
            'text_3': response.xpath('string(body/div/div[2]/div/div/div/div/div/p[5])').extract_first(),
            'text_4': response.xpath('string(body/div/div[2]/div/div/div/div/div/p[6])').extract_first(),
            'text_5': response.xpath('string(body/div/div[2]/div/div/div/div/div/p[7])').extract_first(),
            'text_6': response.xpath('string(body/div/div[2]/div/div/div/div/div/p[8])').extract_first()
        }
            
        page = response.url.split("/")[-1]
#         filename = 'case_%s' % page
#         with open(filename, 'wb') as f:
#             f.write(response.body)
#         self.log('Saved file %s' % filename)
        
# Instantiate our crawler.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'test.json',       # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(CFRSpider)
process.start()
print("Done! It took " + str(time.time() - t0) + " seconds." )

Running...
Done! It took 32.78656506538391 seconds.


In [3]:
import pandas as pd

test = pd.read_json('test.json', orient='records')
print(test.shape)
test.head().T

(165, 23)


,0,1,2,3,4
case,Cas 2015-007N,Cas 2015-008N,Cas 2015-002N,Cas 2015-001N,Cas 2015-005N
name,Antisemitischer Post auf Facebook: «was wotsch...,Antisemitischer Facebook Eintrag: „erst jetzt ...,Rassismus unter Restaurantgästen: Hitlergruss ...,Antisemitischer Facebook-Eintrag: «Diese Kinde...,Flüchtlige aus Eritrea und Syrien haben einen ...
location,Berne,St-Gall,Argovie,Zurich,St-Gall
year,2015,2015,2015,2015,2015
link,/f524/2015-007N.html#2015-007N,/f524/2015-008N.html#2015-008N,/f524/2015-002N.html#2015-002N,/f524/2015-001N.html#2015-001N,/f524/2015-005N.html#2015-005N
history,Die zuständige Strafverfolgungsbehörde verurte...,Die zuständige Strafverfolgungsbehörde verurte...,Die zuständige Strafverfolgungsbehörde verurte...,Die zuständige Strafverfolgungsbehörde verurte...,Die zuständige Strafverfolgungsbehörde verurte...
authors,Particuliers,Particuliers,Particuliers,Particuliers,Particuliers
victims,Juifs,Juifs,Etrangers / autres ethnies,Juifs,Etrangers / autres ethnies;\nRequérants d'asile
means,Ecrits;\nCommunication électronique,Ecrits;\nCommunication électronique,Déclarations orales;\nGestes,Ecrits;\nCommunication électronique;\nSons / i...,Ecrits;\nCommunication électronique
social_env,Médias sociaux,Médias sociaux,Lieux publics,Médias sociaux,Mass media (Internet inclus)


In [4]:
test

,case,name,location,year,link,history,authors,victims,means,social_env,...,title_3,title_4,title_5,title_6,text_1,text_2,text_3,text_4,text_5,text_6
0,Cas 2015-007N,Antisemitischer Post auf Facebook: «was wotsch...,Berne,2015,/f524/2015-007N.html#2015-007N,Die zuständige Strafverfolgungsbehörde verurte...,Particuliers,Juifs,Ecrits;\nCommunication électronique,Médias sociaux,...,None,None,None,None,Der Beschuldigte verfasste und verbreitete auf...,Die zuständige Strafverfolgungsbehörde verurte...,,,,
1,Cas 2015-008N,Antisemitischer Facebook Eintrag: „erst jetzt ...,St-Gall,2015,/f524/2015-008N.html#2015-008N,Die zuständige Strafverfolgungsbehörde verurte...,Particuliers,Juifs,Ecrits;\nCommunication électronique,Médias sociaux,...,None,None,None,None,Die Beschuldigte postete auf einer öffentliche...,Die zuständige Strafverfolgungsbehörde verurte...,,,,
2,Cas 2015-002N,Rassismus unter Restaurantgästen: Hitlergruss ...,Argovie,2015,/f524/2015-002N.html#2015-002N,Die zuständige Strafverfolgungsbehörde verurte...,Particuliers,Etrangers / autres ethnies,Déclarations orales;\nGestes,Lieux publics,...,None,None,None,None,Der Beschuldigte befand sich in einem Restaura...,Die zuständige Strafverfolgungsbehörde verurte...,,,,
3,Cas 2015-001N,Antisemitischer Facebook-Eintrag: «Diese Kinde...,Zurich,2015,/f524/2015-001N.html#2015-001N,Die zuständige Strafverfolgungsbehörde verurte...,Particuliers,Juifs,Ecrits;\nCommunication électronique;\nSons / i...,Médias sociaux,...,None,None,None,None,Der Beschuldigte postete auf einer öffentliche...,Die zuständige Strafverfolgungsbehörde verurte...,,,,
4,Cas 2015-005N,Flüchtlige aus Eritrea und Syrien haben einen ...,St-Gall,2015,/f524/2015-005N.html#2015-005N,Die zuständige Strafverfolgungsbehörde verurte...,Particuliers,Etrangers / autres ethnies;\nRequérants d'asile,Ecrits;\nCommunication électronique,Mass media (Internet inclus),...,None,None,None,None,Der Beschuldigte betrieb eine öffentlich zugän...,Der Beschuldigte wird der Rassendiskriminierun...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Cas 2018-034N,Insultes à caractère raciste envers une femme ...,Jura,2018,/f524/2018-034N.html#2018-034N,L’autorité de poursuite pénale compétente décl...,Particuliers,Noirs / personnes de couleur,Déclarations orales,Aucune indication sur l'environnement social,...,None,None,None,None,Le prévenu a traité devant les personnes prése...,Le prévenu est condamné à une peine pécuniaire...,,,,
161,Cas 2018-037N,Tatbestand unbekannt,Bâle-Ville,2018,/f524/2018-037N.html#2018-037N,Der Beschuldigte wird der Rassendiskriminierun...,Aucune indication sur l'auteur,Aucune indication sur la victime,Aucune indication sur les moyens utilisés,Aucune indication sur l'environnement social,...,None,None,None,None,Tatbestand unbekannt\n,Der Beschuldigte wird der Rassendiskriminierun...,,,,
162,Cas 2018-036N,Empfehlung einer Untersuchung beim Joseph Mengele,Zurich,2018,/f524/2018-036N.html#2018-036N,Die zuständige Strafverfolgungsbehörde verfügt...,Particuliers,Aucune indication sur la victime,Ecrits;\nCommunication électronique,Mass media (Internet inclus);\nMédias sociaux,...,None,None,None,None,Der Beschuldigte postete die folgenden Beiträg...,Die zuständige Strafverfolgungsbehörde verfügt...,,,,
163,Cas 2018-030N,Service de l'aide sociale,Fribourg,2018,/f524/2018-030N.html#2018-030N,L’autorité de poursuite pénale compétente reco...,Particuliers,Noirs / personnes de couleur,Déclarations orales,Lieux publics;\nAutorités / administration / a...,...,None,None,None,None,"Selon les faits établis, le prévenu s’est rend...",Le prévenu est condamné à une peine privative ...,,,,


In [ ]:
# some cleaning 

In [6]:
# save data table as .csv

test.to_csv(path_or_buf='test.csv')